## AB TEST

In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, \
    f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### İş Problemi
Facebook kısa süre önce mevcut maximum bidding adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan average bidding’i tanıttı.
Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi ve averagebidding’in, maximumbidding’den daha fazla dönüşüm
getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.
Maximum Bidding: Maksimum teklif verme
Average Bidding: Average teklif verme

### Veri Seti Hikayesi
bombabomba.com’un web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra 
buradan gelen kazanç bilgileri yer almaktadır.
Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri ab_testing.xlsx excelinin ayrı sayfalarında yer almaktadır.

* Impression – Reklam görüntüleme sayısı
* Click – Tıklama
* Görüntülenen reklama tıklanma sayısını belirtir.
* Değişkenler
* Purchase – Satın alım
* Tıklanan reklamlar sonrası satın alınan ürün sayısını belirtir.
* Earning – Kazanç
* Satın alınan ürünler sonrası elde edilen kazanç

### Görev 1:
A/B testinin hipotezini tanımlayınız.

* H0: Control ve Test Grouplarının ortalaması  arasında fark yoktur.
* H1: ... vardır.

In [4]:
# Descriptive Statistics (Betimsel İstatistikler)


ctrl_group = pd.read_excel("ab_testing.xlsx", sheet_name= "Control Group")
test_group = pd.read_excel("ab_testing.xlsx", sheet_name= "Test Group")

In [5]:
ctrl_group.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522


In [6]:
test_group.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971


In [7]:
ctrl_group.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [8]:
test_group.isnull().sum()

Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64

In [9]:
ctrl_group["Purchase"].mean()

np.float64(550.8940587702316)

In [10]:
test_group["Purchase"].mean()

np.float64(582.1060966484677)

### Varsayım sağlanıyorsa pearson sağlanmıyorsa Spearman.

* H0: Normal dağılım varsayımı sağlanmaktadır.
* H1:..sağlanmamaktadır.

In [11]:
test_stat, pvalue = shapiro(ctrl_group["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [12]:
test_stat, pvalue = shapiro(test_group["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


H0 reddedilemez. p-value > 0.05(p-value(ctrl) = 0.5891, p-value(test) = 0.1541). Dağılımlar Normaldir.

* Varyans homojenliği
* H0: Varyanslar Homojendir
* H1: Varyanslar Homojen Değildir

In [13]:
test_stat, pvalue = levene(ctrl_group["Purchase"],
                           test_group["Purchase"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


H0 reddilemez. p-value >0.05(p-value = 0.1083). Varyanslar Homojendir.

### Görev 2:
Hipotez testini gerçekleştiriniz. Çıkan sonuçların istatistiksel olarak anlamlı olup olmadığını yorumlayınız.

In [14]:
test_stat, pvalue = ttest_ind(ctrl_group["Purchase"],
                              test_group["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


H0 reddedilemez(p-value = 0.3493). Ortalamalar arasında istatistiksel olarak anlamlı bir fark yoktur.

### Görev 3:
Hangi testi kullandınız, sebeplerini belirtiniz.

* Normallik dağılım testi için shapiro-wilks testi yaptık. Dağılımlar normal dağılıyor.
* Varyans homojenliğini levene ile  test ettik. varyanslar homojendir.
* Varsayımlarımız sağlandığı için Bağımsız İki Örneklem T-test'ini uyguladık. Varyansların homojen dağıldığını da belirttik.



### Görev 4:
Görev 2’de verdiğiniz cevaba göre, müşteriye tavsiyeniz nedir?

* iki uygulama arasında fark yoktur. Kontrol ve Test gruplarının sayısı 40'ar olduğu için deneyin sürdürülüp daha fazla katılımcı ile gerçekleştirilmesini öneririm.